In [0]:
! nvidia-smi

Mon May 25 11:56:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import os
import pandas as pd
import numpy as np
os.chdir('/content/drive/My Drive/tabditor/FromZero/rsc')

# 데이터 준비

## label 1에 해당하는 데이터 불러오기

In [2]:
label = pd.read_csv('label_1.csv')
print(label.shape)
label.head()

(5541, 7)


,index,qus,ans,prep_qus,label,prepro_label,prepro_ans
0,9269,[Olive Young과 함께 마음까지 All live young하는 삶을 꿈꾸다],\n제가 올리브영에 입사하고 싶은 이유는 올리브영만의 기업철학이 제가 지향하는 삶의...,OliveYoung과함께마음까지Allliveyoung하는삶을꿈꾸다,[1],1,제가 올리브영에 입사하고 싶은 이유는 올리브영만의 기업철학이 제가 지향하는 삶의 방...
1,17155,경쟁과 협력 가운데 더 중요하다고 생각하는 것,\r\n무한경쟁시대일수록 역설적으로 협력의 힘이 더욱 중요하다고 생각합니다. 함께 ...,경쟁과협력가운데더중요하다고생각하는것,[1],1,무한경쟁시대일수록 역설적으로 협력의 힘이 더욱 중요하다고 생각합니다 함께 힘을 합친...
2,9623,[고퀄리티 제품과 서비스를 갖춘 GS엠비즈의 고퀄리티 영업관리자가 되겠습니다],"\r\nGS엠비즈는 ""The Best Company for Customers""를 비...",고퀄리티제품과서비스를갖춘GS엠비즈의고퀄리티영업관리자가되겠습니다,[1],1,GS엠비즈는 The Best Company for Customers를 비전으로 자동...
3,4410,공공기관으로서 사업 추진에 필요한 영화계 의견수렴 방법을 제시하여 주십시오 1000바이트,"\r\n""현장 상황을 존중하는 적극적인 자세와 직접 소통의 중요성""\r 저는 영화진...",공공기관으로서사업추진에필요한영화계의견수렴방법을제시하여주십시오1000바이트,"[1,2]","1,2",현장 상황을 존중하는 적극적인 자세와 직접 소통의 중요성 저는 영화진흥위원회의 의견...
4,7456,공공기관으로써 갖추어야 할 덕목,\r\n한국토지공사는 사기업과는 다른 공기업이기 때문에 개인보다는 국민을 먼저 생각...,공공기관으로써갖추어야할덕목,[1],1,한국토지공사는 사기업과는 다른 공기업이기 때문에 개인보다는 국민을 먼저 생각하는 배...


# text preprocessing

## train을 위한 answer 데이터 전처리

In [3]:
label.prepro_ans

0       제가 올리브영에 입사하고 싶은 이유는 올리브영만의 기업철학이 제가 지향하는 삶의 방...
1       무한경쟁시대일수록 역설적으로 협력의 힘이 더욱 중요하다고 생각합니다 함께 힘을 합친...
2       GS엠비즈는 The Best Company for Customers를 비전으로 자동...
3       현장 상황을 존중하는 적극적인 자세와 직접 소통의 중요성 저는 영화진흥위원회의 의견...
4       한국토지공사는 사기업과는 다른 공기업이기 때문에 개인보다는 국민을 먼저 생각하는 배...
                              ...                        
5536    식품 연구원으로의 성장저의 지도교수님은 한 분야의 전문가가 되려면 최소 10년의 지...
5537    실험을 통한 분석력과 조별과제에서 배운 소통능력 생산 관리 직무는 현장에서 생산 계...
5538    저는  태권도 동아리 소속의 아낌없이 가르쳐주고 소통하는 멘토입니다에 입학 후 건강...
5539    병원에 입원해 본 사람은 매 시간이 얼마나 감사한지 알 것입니다 사고로 군용 차량과...
5540    병원에 입원해 본 사람은 매 시간이 얼마나 감사한지 알 것입니다 사고로 군용 차량과...
Name: prepro_ans, Length: 5541, dtype: object

### keras tokenizer로 어절 단위의 tokenizing을 진행한다.  
keras tokenizer 장점
- word to index를 진행해준다.
- word to sequence를  진행해준다. 


In [3]:
from keras.preprocessing.text import Tokenizer

token = Tokenizer()
token.fit_on_texts(label.prepro_ans)

Using TensorFlow backend.


In [4]:
# 어절단위 tokenizing 진행 후, 단어의 개수 124385개  
len(token.word_index)

124385

In [0]:
slide_data = []

for i in label.prepro_ans:
    seq_data = token.texts_to_sequences([i])
    for j in range(1,len(seq_data[0])):
        tmp = seq_data[0][0:j+1]
        slide_data.append(tmp)

        

In [7]:
len(slide_data)

785537

In [6]:
slide_data[:10]

[[20, 24506],
 [20, 24506, 6076],
 [20, 24506, 6076, 201],
 [20, 24506, 6076, 201, 341],
 [20, 24506, 6076, 201, 341, 51179],
 [20, 24506, 6076, 201, 341, 51179, 51180],
 [20, 24506, 6076, 201, 341, 51179, 51180, 20],
 [20, 24506, 6076, 201, 341, 51179, 51180, 20, 4205],
 [20, 24506, 6076, 201, 341, 51179, 51180, 20, 4205, 450],
 [20, 24506, 6076, 201, 341, 51179, 51180, 20, 4205, 450, 9291]]

In [7]:
maxlen = max([len(i) for i in slide_data])
maxlen

975

In [8]:
from keras.preprocessing.sequence import pad_sequences

seq_data = pad_sequences(slide_data,maxlen=maxlen)
seq_data

array([[    0,     0,     0, ...,     0,    20, 24506],
       [    0,     0,     0, ...,    20, 24506,  6076],
       [    0,     0,     0, ..., 24506,  6076,   201],
       ...,
       [    0,     0,     0, ..., 13138,  1143,   450],
       [    0,     0,     0, ...,  1143,   450,   249],
       [    0,     0,     0, ...,   450,   249,   284]], dtype=int32)

In [0]:
x_train = seq_data[:,:-1]
y_train = seq_data[:,-1]

In [11]:
max_y = max([i for i in y_train])
max_y

124385

In [0]:
# from keras.utils import to_categorical
# y_train = to_categorical(y_train, num_classes=max_y+1)

# LSTM 적용

In [15]:
from keras.layers import Dense,Embedding,LSTM
from keras.models import Sequential
from keras.models import load_model


model = Sequential()
model.add(Embedding(max_y+1,10,input_length=maxlen-1))
model.add(LSTM(64))
model.add(Dense(max_y+1,activation='softmax'))
# model.add(Dense(1,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer = 'adam',metrics=['accuracy']) #metrics = 평가척도

# model.load_weights('ckpt_epoch200.h5')
model.fit(x_train,y_train,epochs = 100,verbose=2)

# model.save('ckpt_epoch100.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100


KeyboardInterrupt: ignored